In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('BD Артикулы - Лист1.csv')

In [3]:
df

,product_id,seller_id,mp_id,category,marketplace_name,legal_entity
0,AT-001W,AT-001W,1.364959e+09,Прочее,OZON,ИНТЕР
1,AT-002BE,AT-002BE,1.364959e+09,Прочее,OZON,ИНТЕР
2,AT-002BK,AT-002BK,1.364959e+09,Прочее,OZON,ИНТЕР
3,AT-002G,AT-002G,1.364959e+09,Прочее,OZON,ИНТЕР
4,AT-003BK,AT-003BK,1.364959e+09,Прочее,OZON,ИНТЕР
...,...,...,...,...,...,...
498,Н-МОР3-1,H-MOP3-1,2.870870e+08,Швабры,Wildberries,КРАВЧИК
499,CHS1-1C,CHS1-1C,NaN,Инструменты,Wildberries,КРАВЧИК
500,FP1-4,FP1-4,3.307301e+08,Сковороды,Wildberries,АТ
501,FP1-5,FP1-5,3.331718e+08,Сковороды,Wildberries,АТ


In [4]:
dbname='app_db'
user='user'
host='db'
port=5432

In [5]:
import getpass
password = getpass.getpass()

 ········


In [6]:
from psycopg2 import connect, sql
from functools import wraps
from typing import Callable
from datetime import datetime, timedelta

def inject_closable_cursor(func: Callable):
    @wraps(func)
    def wrapper(*args, **kwargs):
        connection = connect(
            dbname=dbname,
            user=user,
            password=password,
            host=host,
            port=port,
            sslmode="require"
        )
        try:
            with connection.cursor() as cursor:
                result = func(*args, cursor=cursor, **kwargs)
                connection.commit()
                return result
        except Exception:
            connection.rollback()
            raise
        finally:
            connection.close()
    return wrapper

In [7]:
import numpy as np

@inject_closable_cursor
def add_data(data, **kwargs):
    cursor = kwargs.get('cursor')

    try:
        for _, record in data.iterrows():
            cursor.execute(
                sql.SQL('''
                    INSERT INTO sku (product_id, seller_id, mp_id, category, marketplace_name, legal_entity)
                    VALUES (%s, %s, %s, %s, %s, %s)
                '''),
                [record['product_id'], record['seller_id'], None if pd.isna(record['mp_id']) else int(record['mp_id']), record['category'], record['marketplace_name'],
           record['legal_entity']]
            )
    except:
        print(type(record['mp_id']))
        raise

add_data(df)

In [8]:
df.columns

Index(['product_id', 'seller_id', 'mp_id', 'category', 'marketplace_name',
       'legal_entity'],
      dtype='object')

In [9]:
df.mp_id.values[0].dtype, df.mp_id.values[-1].dtype

(dtype('float64'), dtype('float64'))